In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
#tf.enable_eager_execution()
tf.compat.v1.enable_eager_execution()

import tensorflow_hub as hub
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
#from keras import optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.15.0
Eager mode:  True
Hub version:  0.16.1
GPU is available


In [ ]:
#root_dir = "/content/drive/MyDrive/Kaggle_Datasets/plantdisease/PlantVillage"

train_dir = "/content/drive/MyDrive/processed_data/train"
val_dir = "/content/drive/MyDrive/processed_data/val"
test_dir = "/content/drive/MyDrive/processed_data/test"

In [ ]:
import time
import os
from os.path import exists

def count(dir, counter=0):
    "returns number of files in dir and subdirs"
    for pack in os.walk(dir):
        for f in pack[2]:
            counter += 1
    return dir + " : " + str(counter) + "files"

In [ ]:
print('total images for training :', count(train_dir))
print('total images for validation :', count(val_dir))
print('total images for validation :', count(test_dir))

total images for training : /content/drive/MyDrive/processed_data/train : 11464files
total images for validation : /content/drive/MyDrive/processed_data/val : 1431files
total images for validation : /content/drive/MyDrive/processed_data/test : 1437files


In [ ]:
IMAGE_SHAPE = (224, 224)

BATCH_SIZE = 64

In [ ]:
# Create an ImageDataGenerator for validation data (no data augmentation)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    shuffle=False,
    seed=42,
    color_mode="rgb",
    class_mode="categorical",
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE
)

do_data_augmentation = False
if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )
else:
    train_datagen = validation_datagen  # Use the same normalization for training data

train_generator = train_datagen.flow_from_directory(
    train_dir,
    subset="training",
    shuffle=True,
    seed=42,
    color_mode="rgb",
    class_mode="categorical",
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 1431 images belonging to 5 classes.
Found 11464 images belonging to 5 classes.
Found 1437 images belonging to 5 classes.


In [ ]:
# Compute the number of classes dynamically
num_classes = len(train_generator.class_indices)

In [ ]:
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense

In [ ]:
base =  tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape= list(IMAGE_SHAPE)+[3],
    include_top=False,
    weights='imagenet',
)

# Add a global spatial average pooling layer
out = base.output
out = GlobalAveragePooling2D()(out)
out=Dropout(0.4)(out)
out = Dense(512, activation='relu')(out)
out=Dropout(0.2)(out)
predictions = Dense(5, activation='softmax')(out)
model = Model(inputs=base.input, outputs=predictions)

# only if we want to freeze layers
for layer in base.layers[:-1]:
    layer.trainable = False
base.layers[-1].trainable =True

# Compile
#opt = Adam(lr=learning_rate, decay=learning_rate / EPOCHS)

LEARNING_RATE = 0.1
model.compile(
   optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),loss='categorical_crossentropy',metrics=["accuracy"])


In [ ]:
# Define callbacks
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.01,
    patience=3,
    factor=0.25,
    verbose=1,
    cooldown=0,
    min_lr=0.00000001
)

early_stopper = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.005,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

EPOCHS = 30

# Define callbacks
checkpoint_path = '/content/drive/MyDrive/mobilenet_model_checkpoint.h5'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# Attempt to train the model
model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    #callbacks=[reduce_lr, early_stopper, checkpoint_callback]
)



8/8 [==============================] - 279s 32s/step - loss: 273.8210 - accuracy: 0.2285 - val_loss: 40.5978 - val_accuracy: 0.2180


In [ ]:
# Example usage: Evaluate model on test data
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 6))
plt.plot(epochs_range, acc, label='Training Accuracy', color='blue')
plt.plot(epochs_range, val_acc, label='Validation Accuracy', color='orange')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Plot Training and Validation Loss
plt.figure(figsize=(8, 6))
plt.plot(epochs_range, loss, label='Training Loss', color='blue')
plt.plot(epochs_range, val_loss, label='Validation Loss', color='orange')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pylab as plt
import numpy as np

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

#plt.savefig('/content/drive/MyDrive/after_30_epoch_accuracy.png')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()
#plt.savefig('/content/drive/MyDrive/after_30_epoch_loss.png')

In [ ]:
# Import OpenCV
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob

def load_image(filename):
    img = cv2.imread(os.path.join(root_dir, val_dir, filename))
    assert not isinstance(img,type(None)), 'image not found'

    img = cv2.resize(img, (IMAGE_SHAPE[0], IMAGE_SHAPE[1]) )

    img = img /255


    return img


def predict(image):
    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)

    return {classes[class_idx]: probabilities[class_idx]}

In [ ]:
tf.saved_model.save(model, "/content/drive/MyDrive/Models/mobileNet_saved_on_2_epoch")

In [ ]:
import matplotlib.pyplot as plt
def display_batch(generator):
    images, labels = next(generator)
    for i in range(len(images)):
        plt.imshow(images[i])
        plt.title(f"Label: {np.argmax(labels[i])}")
        plt.axis('off')
        plt.show()

# Display the first batch of test images and their labels
display_batch(test_generator)

NameError: name 'plt' is not defined

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/MyDrive/efficientnet_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/mobilenetNet.h5")
print("Saved model to disk")